In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use(['fivethirtyeight', 'figsize'])
_SAVEDIR_ = 'Tex/figures/'


def prog_bar(cur_val,max_val,msg = ""):
    import sys
    sys.stdout.write('\r')
    eq = int(np.ceil(np.true_divide(cur_val*100,max_val*5)))
    sys.stdout.write("[{:20s}] {}/{} {:}  ".format('='*eq, cur_val+1,max_val,msg))
    sys.stdout.flush()


In [32]:
# Define parameters
G = -5
# omega = 1.d0/(3.*visc*(dt*dt)/(dx*dx) + 0.5*dt)
omega = 1

In [ ]:
def init(rho_in):
    rho = np.ones([lx,ly])*rho_in
    
    rho *= 1+.01*(np.random.rand(lx,ly)-.5)*2
    u = np.zeros([lx,ly])
    v = np.zeros([lx,ly])
    for kk in range(0,9):  
        f[0,kk,:,:] = w[kk]*rho
        
    # Init rho with perturbation
def calc_macro(f,rho,u,v):
    # f should be at a given time step
    u[:,:] = 0
    v[:,:] = 0
    
    rho = np.sum(f,axis=0)
    for k in range(9):
        u += f[k]*ex[k]
#         print(ey[k])
        v += f[k]*ey[k]
    u=u/rho
    v=v/rho
    return rho, u ,v
def force(rho, u, v):
    # omega, and rho_psi are global variables
    psi = rho_psi*(1-np.exp(-rho/rho_psi))
    # a negative roll velocity implied i+1

    f_x = psi*(np.roll(psi,(0,-1),(0,1))-np.roll(psi,(0,1),(0,1)))
    f_y = psi*(np.roll(psi,(0,-1),(0,1))-np.roll(psi,(0,1),(0,1)))

    f2_x = psi*(np.roll(psi,(-1,1),(0,1))+np.roll(psi,(-1,-1),(0,1))
                -np.roll(psi,(1,1),(0,1))-np.roll(psi,(1,-1),(0,1)))


    f2_y = psi*(np.roll(psi,(-1,-1),(0,1))+np.roll(psi,(1,-1),(0,1))
                -np.roll(psi,(1,1),(0,1))-np.roll(psi,(-1,1),(0,1)))

    force_x = -G*(f_x*1/9+f2_x*1/36)
    force_y = -G*(f_y*1/9+f2_y*1/36)


    # shift equlibrium
    u += force_x/(omega*rho)
    v += force_y/(omega*rho)
    return u, v

def equilibrium(u, v, rho, f, ts):
    for k in range(0,9):
        cu = (1./cs**2)*(ex[k]*u+ey[k]*v)
        feq = rho*w[k]*(1.+cu+cu**2-(u**2+v**2)/(2*cs**2))
        f[ts, k, :,:] = f[ts-1,k,:,:] +(1./tau)*(feq-f[ts-1,k,:,:])

def run(f,boundaries,internal_bound):
# initialization
    rho = np.ones([lx,ly])
    u = np.zeros([lx,ly])
    v = np.zeros([lx,ly])
    for kk in range(0,9):  
        f[0,kk,:,:] = w[kk]*rho

    # main loop
    for ts in range(1,Nsteps):
        # macroscopic
        prog_bar(ts,Nsteps)
        rho, u, v = calc_macro(f[ts-1],rho,u,v)

        # collision
        force()
        equilibrium(u, v, rho, f, ts)
#         for k in range(0,9):
#             cu = (1./cs**2)*(ex[k]*u+ey[k]*v)
#             feq = rho*w[k]*(1.+cu+cu**2-(u**2+v**2)/(2*cs**2))
#             f[ts, k, :,:] = f[ts-1,k,:,:] +(1./tau)*(feq-f[ts-1,k,:,:])


        # prepare the next time step
        fout = f[ts]

        # boundary
        # north & south
        for k in range(0,9):
            fout[k][boundaries == 1] = f[ts,opp[k]][boundaries == 1]
#             fout[k][internal_bound==1]=0
        # streaming
        for kk in range(0,9): 
            f[ts,kk,:,:] = np.roll(fout[kk,:,:],(ex[kk],ey[kk]),(0,1))

In [13]:
psi = np.arange(1,5)
print(psi)
print(psi*(np.roll(psi,-1,0)-np.roll(psi,1,0)))
for i in range(4):
    print(psi[i]*(psi[(i+1)%4]-psi[i-1]))

[1 2 3 4]
[-2  4  6 -8]
-2
4
6
-8


In [ ]:
psi = np.arange(1,5)
print(psi)
print(psi*(np.roll(psi,-1,0)-np.roll(psi,1,0)))
for i in range(4):
    print(psi[i]*(psi[(i+1)%4]-psi[i-1]))

In [29]:
psi =np.zeros([5,5])
psi[:,:] = np.arange(1,6)
psi[3,4] = 4
print(psi*(np.roll(psi,(-1,0),(0,1))-np.roll(psi,(1,0),(0,1))))
for i in range(5):
    print(psi[i,:]*(psi[(i+1)%4,:]-psi[i-1,:]))

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -5.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  5.]]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0. -5.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  5.]


In [33]:
# a negative roll velocity implied i+1

f_x = psi*(np.roll(psi,(0,-1),(0,1))-np.roll(psi,(0,1),(0,1)))
f_y = psi*(np.roll(psi,(0,-1),(0,1))-np.roll(psi,(0,1),(0,1)))

f2_x = psi*(np.roll(psi,(-1,1),(0,1))+np.roll(psi,(-1,-1),(0,1))
            -np.roll(psi,(1,1),(0,1))-np.roll(psi,(1,-1),(0,1)))


f2_y = psi*(np.roll(psi,(-1,-1),(0,1))+np.roll(psi,(1,-1),(0,1))
            -np.roll(psi,(1,1),(0,1))-np.roll(psi,(-1,1),(0,1)))

force_x = -G*(f_x*1/9+f2_x*1/36)
force_y = -G*(f_y*1/9+f2_y*1/36)


# shift equlibrium
u += force_x/(omega*rho)
v += force_y/(omega*rho)

NameError: name 'u' is not defined